In [0]:
# Libraries 
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [0]:
# Training Data preprocessing
training_data = pd.read_excel('/content/training_input.xlsx')

for i in range(training_data.shape[0]):
  if training_data.loc[i, 'Type'] == 1:
    break
  training_data.loc[i, 'Type'] = 0

for i in range(training_data.shape[0]):
  if training_data.loc[i, 'Type'] == 0:
    continue
  else:
    training_data.loc[i, 'Type'] = 1

training_data = training_data.sample(n=len(training_data))

training_d = []
training_labels = []
for i in range(training_data.shape[0]):
  training_d.append([training_data.loc[i, 'Data 1'],training_data.loc[i, 'Data 2'], training_data.loc[i, 'Data 3'], training_data.loc[i, 'Data 4'], training_data.loc[i, 'Data 5']])
  training_labels.append(training_data.loc[i, 'Type'])

# Test Data preprocessing
test_data = pd.read_excel('/content/test_input.xlsx')

for i in range(test_data.shape[0]):
  if test_data.loc[i, 'Type'] == 1:
    break
  test_data.loc[i, 'Type'] = 0

for i in range(test_data.shape[0]):
  if test_data.loc[i, 'Type'] == 0:
    continue
  else:
    test_data.loc[i, 'Type'] = 1

test_data = test_data.sample(n=len(test_data))

test_d = []
test_labels = []
for i in range(test_data.shape[0]):
  test_d.append([test_data.loc[i, 'Data 1'], test_data.loc[i, 'Data 2'], test_data.loc[i, 'Data 3'], test_data.loc[i, 'Data 4'], test_data.loc[i, 'Data 5']])
  test_labels.append(test_data.loc[i, 'Type'])


In [0]:
# Dictionary containing all the ML models that need to be implemented
models = {
    "svm": SVC(kernel="poly")
}

In [18]:
# Conversion of data from type-list to type-numpy array. This makes the further 
# vectorization of the process possible. 
trainX = np.array(training_d)
trainY = np.array(training_labels)

testX = np.array(test_d)
testY = np.array(test_labels)

# Training SVM model of the training data
print("---------------------------------------------Support Vector Machines-------------------------------------------")
model = models["svm"]
model.fit(trainX, trainY)

# Testing the performance of model on test data
predictions = model.predict(testX)
print(classification_report(testY, predictions))

# Additional evaluation matrices for better performace testing
pred_Y = model.predict(testX)
labels_nn = [0, 1]
num_examples = len(pred_Y)
C_values = np.zeros((2, 2, num_examples))
for i in range(2):
  C_values[0][:][i] = (pred_Y == labels_nn[i])
  C_values[1][:][i] = (testY == labels_nn[i])
preformance_measures = np.zeros((2, 6))
for i in range(2):
  TP = sum(((C_values[0][:][i] == 1) & (C_values[1][:][i] == 1)))
  FP = sum(((C_values[0][:][i] == 1) & (C_values[1][:][i] == 0)))
  TN = sum(((C_values[0][:][i] == 0) & (C_values[1][:][i] == 0)))
  FN = sum(((C_values[0][:][i] == 0) & (C_values[1][:][i] == 1)))
  precision = (TP)/(TP + FP)
  recall = (TP)/(TP + FN)
  f1_score = (2*precision*recall)/(recall + precision)
  specificity = (TN)/(TN + FP)
  ROC = ((recall**2 + specificity**2)**(0.5))/(2**(0.5))
  geometric_mean = (recall*specificity)**(0.5)
  preformance_measures[i][0] = precision
  preformance_measures[i][1] = recall
  preformance_measures[i][2] = f1_score
  preformance_measures[i][3] = specificity
  preformance_measures[i][4] = ROC
  preformance_measures[i][5] = geometric_mean



preformance_measures = pd.DataFrame(preformance_measures, index=labels_nn, columns=['Precision', 'Recall/Sensitivity', 'F1_Score', 'Specificity', 'ROC', 'Geometric Mean'] )
preformance_measures

---------------------------------------------Support Vector Machines-------------------------------------------
              precision    recall  f1-score   support

           0       0.68      0.82      0.75        50
           1       0.78      0.62      0.69        50

    accuracy                           0.72       100
   macro avg       0.73      0.72      0.72       100
weighted avg       0.73      0.72      0.72       100



,Precision,Recall/Sensitivity,F1_Score,Specificity,ROC,Geometric Mean
0,0.683333,0.82,0.745455,0.62,0.726911,0.713022
1,0.775000,0.62,0.688889,0.82,0.726911,0.713022
